# Packages

In [11]:
import os

import lightning
import pandas as pd
import torch
import torchmetrics
from torch.utils.data import DataLoader
from collections import OrderedDict
from torchvision.transforms import Compose
from load_data import AudioTrainDataset, TargetEncoder, PaddingZeros, NormalizedMelSpectogram

# Simple loading

In [12]:
DATA_PATH = os.path.join("tensorflow-speech-recognition-challenge", "train", "audio")
dataset = AudioTrainDataset(DATA_PATH)

labels_list, labels_dict = dataset.find_classes(DATA_PATH)
labels_dict = {idx: name for name, idx in labels_dict.items()}
labels_dict

{1: 'bed',
 2: 'bird',
 3: 'cat',
 4: 'dog',
 5: 'down',
 6: 'eight',
 7: 'five',
 8: 'four',
 9: 'go',
 10: 'happy',
 11: 'house',
 12: 'left',
 13: 'marvin',
 14: 'nine',
 15: 'no',
 16: 'off',
 17: 'on',
 18: 'one',
 19: 'right',
 20: 'seven',
 21: 'sheila',
 22: 'silence',
 23: 'six',
 24: 'stop',
 25: 'three',
 26: 'tree',
 27: 'two',
 28: 'up',
 29: 'wow',
 30: 'yes',
 31: 'zero'}

In [13]:
NUM_WORKERS = 6
BATCH_SIZE = 512

# Article model

In [14]:
transforms = Compose([
    PaddingZeros(16000),
    NormalizedMelSpectogram()
])
features_dataset = AudioTrainDataset(DATA_PATH, transform=transforms,
                                     target_transform=TargetEncoder(class_dict=labels_dict))

In [15]:
gen = torch.Generator().manual_seed(42)
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(features_dataset, [0.7, 0.1, 0.2],
                                                                           generator=gen)
len(train_dataset), len(valid_dataset), len(test_dataset)

(45587, 6512, 13024)

In [16]:
train_dataset_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True,
                                  generator=torch.random.manual_seed(123))
valid_dataset_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
test_dataset_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
len(train_dataset_loader), len(valid_dataset_loader), len(test_dataset_loader)

(90, 13, 26)

In [17]:
class ArticleNet(lightning.LightningModule):
    def __init__(self, target_size):
        super().__init__()
        self.conv_part = torch.nn.Sequential(OrderedDict([
            ('conv1', torch.nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 1), padding='same', groups=1)),
            ('relu1', torch.nn.ReLU()),
            ('bn1', torch.nn.BatchNorm2d(num_features=10)),
            ('conv2', torch.nn.Conv2d(in_channels=10, out_channels=1, kernel_size=(5, 1), padding='same', groups=1)),
            ('relu2', torch.nn.ReLU()),
            ('bn2', torch.nn.BatchNorm2d(num_features=1))
        ]))
        self.lstm_1 = torch.nn.LSTM(input_size=126, hidden_size=64, bidirectional=True, batch_first=True, num_layers=2)
        self.query_1 = torch.nn.Linear(in_features=128, out_features=128)
        self.att_scores = torch.nn.Softmax(dim=-1)
        self.dense_part = torch.nn.Sequential(OrderedDict([
            ('dense1', torch.nn.Linear(in_features=128, out_features=64)),
            ('relu3', torch.nn.ReLU()),
            ('dense2', torch.nn.Linear(in_features=64, out_features=32)),
            ('relu4', torch.nn.ReLU()),
            ('dense3', torch.nn.Linear(in_features=32, out_features=target_size)),
            ('softmax', torch.nn.Softmax(dim=-1))
        ]))
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=target_size)
        self.valid_acc = torchmetrics.Accuracy(task="multiclass", num_classes=target_size)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=target_size)
        self.test_conf_mat = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=target_size)

    def forward(self, x):
        convoluted = self.conv_part(x)
        x_squeezed = convoluted.squeeze()
        x_squeezed = x_squeezed.permute(0, 2, 1)
        lstm_out, _ = self.lstm_1(x_squeezed)
        last = lstm_out[:, -1, :]
        query = self.query_1(last)
        att_scores = torch.bmm(query.unsqueeze(1), lstm_out.transpose(1, 2)).squeeze(1)
        att_scores = self.att_scores(att_scores)
        att_vector = torch.bmm(att_scores.unsqueeze(1), lstm_out).squeeze(1)
        output = self.dense_part(att_vector)
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        self.train_acc(y_hat, torch.argmax(y, dim=-1))
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_acc_step", self.train_acc)
        return loss

    def on_train_epoch_end(self):
        self.log('train_acc', self.train_acc)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        return self(x)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        self.valid_acc(y_hat, torch.argmax(y, dim=-1))
        self.log('val_loss', loss, on_epoch=True)
        self.log('val_acc', self.valid_acc, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        y_class = torch.argmax(y, dim=-1)
        self.test_acc(y_hat, y_class)
        self.test_conf_mat(y_hat, y_class)
        self.log('test_loss', loss, on_epoch=True)
        self.log('test_acc', self.test_acc, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.3, patience=3),
                "monitor": "val_loss",
            }
        }

In [18]:
for batch_x, batch_y in train_dataset_loader:
    print(batch_x.shape)
    print(batch_x.permute(0, 2, 3, 1).shape)
    break

torch.Size([512, 1, 126, 128])
torch.Size([512, 126, 128, 1])


In [19]:
model = ArticleNet(12)
for batch_x, batch_y in train_dataset_loader:
    y_hat = model(batch_x)
    break

In [20]:
model = ArticleNet(12)
trainer = lightning.Trainer(max_epochs=2, logger=True)
torch.set_float32_matmul_precision('high')
trainer.fit(model, train_dataloaders=train_dataset_loader, val_dataloaders=valid_dataset_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | conv_part     | Sequential                | 133   
1 | lstm_1        | LSTM                      | 197 K 
2 | query_1       | Linear                    | 16.5 K
3 | att_scores    | Softmax                   | 0     
4 | dense_part    | Sequential                | 10.7 K
5 | train_acc     | MulticlassAccuracy        | 0     
6 | valid_acc     | MulticlassAccuracy        | 0     
7 | test_acc      | MulticlassAccuracy        | 0     
8 | test_conf_mat | MulticlassConfusionMatrix | 0     
------------------------------------------------------------
225 K     Trainable params
0         Non-trainable params
225 K     Total params
0.900     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 90/90 [00:29<00:00,  3.09it/s, v_num=17, train_loss_step=2.090]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 90/90 [00:27<00:00,  3.27it/s, v_num=17, train_loss_step=1.930, train_loss_epoch=2.100]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 90/90 [00:43<00:00,  2.06it/s, v_num=17, train_loss_step=1.930, train_loss_epoch=1.990]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 90/90 [00:43<00:00,  2.06it/s, v_num=17, train_loss_step=1.930, train_loss_epoch=1.990]


In [21]:
results = []
predictions = []
for i in range(5):
    lightning.pytorch.seed_everything(i)
    model = ArticleNet(12)
    early_stopping = lightning.pytorch.callbacks.EarlyStopping('val_loss', verbose=True)
    logger = lightning.pytorch.loggers.tensorboard.TensorBoardLogger(save_dir="article_net", version=i)
    trainer = lightning.Trainer(max_epochs=200, callbacks=[early_stopping], logger=logger)
    trainer.fit(model, train_dataloaders=train_dataset_loader, val_dataloaders=valid_dataset_loader)
    res = trainer.test(dataloaders=test_dataset_loader, ckpt_path='best')
    test_pred_tensor = torch.cat(trainer.predict(dataloaders=test_dataset_loader, ckpt_path='best'))
    results.append(res[0])
    predictions.append(test_pred_tensor)
torch.save(torch.stack(predictions), "article_net_predictions.ts")
pd.DataFrame(results).to_csv("article_net_lstm_metrics.csv")

Global seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | conv_part     | Sequential                | 133   
1 | lstm_1        | LSTM                      | 197 K 
2 | query_1       | Linear                    | 16.5 K
3 | att_scores    | Softmax                   | 0     
4 | dense_part    | Sequential                | 10.7 K
5 | train_acc     | MulticlassAccuracy        | 0     
6 | valid_acc     | MulticlassAccuracy        | 0     
7 | test_acc      | MulticlassAccuracy        | 0     
8 | test_conf_mat | MulticlassConfusionMatrix | 0     
------------------------------------------------------------
225 K     Trainable params
0         Non-trainable params
225 K     Total params
0.900     Total estimated model

Epoch 0: 100%|██████████| 90/90 [00:27<00:00,  3.23it/s, v_num=0, train_loss_step=1.880]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 90/90 [00:44<00:00,  2.02it/s, v_num=0, train_loss_step=1.880, train_loss_epoch=2.090]

Metric val_loss improved. New best score: 1.976


Epoch 1: 100%|██████████| 90/90 [00:28<00:00,  3.16it/s, v_num=0, train_loss_step=2.040, train_loss_epoch=2.090]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 90/90 [00:44<00:00,  2.01it/s, v_num=0, train_loss_step=2.040, train_loss_epoch=1.990]

Metric val_loss improved by 0.002 >= min_delta = 0.0. New best score: 1.974


...

Restoring states from the checkpoint path at article_net\lightning_logs\version_4\checkpoints\epoch=6-step=630.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at article_net\lightning_logs\version_4\checkpoints\epoch=6-step=630.ckpt


Testing DataLoader 0: 100%|██████████| 26/26 [00:03<00:00,  7.07it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6303746700286865     │
│         test_loss         │    1.9883546829223633     │
└───────────────────────────┴───────────────────────────┘

Restoring states from the checkpoint path at article_net\lightning_logs\version_4\checkpoints\epoch=6-step=630.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at article_net\lightning_logs\version_4\checkpoints\epoch=6-step=630.ckpt


Predicting DataLoader 0: 100%|██████████| 26/26 [00:03<00:00,  7.04it/s]
